## Week 1: Overview and Big O

### Intro to Algorithmic thinking

We start by comparing two algorithms for the multiplication of integers: the grade school way (adding partial products) and the Katsubara algorithm. 

The grade school way is $n^2$ complex. For each digit of x, we multiply by each digit of y $(n^2)$. Then, to add the partial products, we perform approximately $n^2$ additions of 2 numbers. $2n^2 \approx n^2$.

Katsubara multiplication breaks x and y into two numbers each, a (first 1/2 of digits) and b (last 1/2 of digits), and c and d. We can say:

$$ x \cdot y = 10^{n}ac + 10^{n/2}(ad + bc) + bd$$

This can be performed recursively as well, if for instance we are limited to multiplying only single-digits together at a time. $ac, ad, bc, bd$ can all be calculated with the same formula. However, we can reduce the number of recursive calls from 4 to 3 by restating $(ad + bc)$ as $(a + b) \cdot (c + d) - ac - bd$ (factor it out and you'll see that these are equivalent hehe). Since we already know ac and bd, it's straightforward. Thus, our Katsubara algorithm:

$$ x \cdot y = 10^{n}ac + 10^{n/2}[(a+b)(c+d) - ac - bd] + bd$$

Hahahahaha loljk I can't keep up this level of note quality in Jupyter while taking notes on paper as well. This can just be a scrapbook for code snippets.

### Merge sort as a Divide and Conquer Algorithm

In [3]:
# Merge sort main function.
def mergeSort(arr):
    if len(arr) < 2:
        return arr
    mid = len(arr) // 2
    merged = merge(mergeSort(arr[:mid]), mergeSort(arr[mid:]))
    print(merged)
    return merged

# Merge helper that merges already-sorted sub-arrays.
def merge(first, second):
    merged = []
    # while at least one of these aren't empty. Yay for falsey values in Python!
    while first or second:
        if first and ((not second) or (second and first[0] < second[0])):
            merged.append(first[0])
            first = first[1:]
        # in all other cases: this helps take care of duplicate items as well.
        else:
            merged.append(second[0])
            second = second[1:]
    return merged

arr = [5, 3, 8, 9, 1, 7, 0, 2, 6, 4]
mergeSort(arr)
            

[3, 5]
[1, 9]
[1, 8, 9]
[1, 3, 5, 8, 9]
[0, 7]
[4, 6]
[2, 4, 6]
[0, 2, 4, 6, 7]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

### Programming Assignment 1: Implementing Karatsuba division
02/05/17

My assignment was to implement karatsuba division in a programming language of my choice, then run it on two obscenely large numbers (64 digits each, to be exact). I did it incorrectly for a couple hours before throwing in the towel (sorry sorry) and editing some other person's code until I got the right answer (very very sorry sorry). I submitted at least like, 5 or 6 times before I got the right answer. Really quite frustrating. Anyways. I want to document some of the mistakes I made over the course of trying to solve this problem, and contrast my original solution with the final one I arrived at. From least complex to most complex:

- I spelled *karatsuba* as *katsubara* (darn metathesis. Although I think I switched around three syllables, ha!). Not technically relevant, but whoops.
- Some of the mathematical operations I performed (namely, / division, which yields a float) gave me floats instead of integers. It turns out that floating point arithmetic in Python is by necessity imprecise (see https://docs.python.org/2/tutorial/floatingpoint.html), and integer operations are extremely precise. I needed to switch out my / for //. (At the *very* beginning, I used divmod for a,b and c,d pairs because I thought I was being clever. But alas, divmod uses the / operator as well.) Also! It was due to this oversight that I kept getting an error "maximum recursion depth exceeded while getting the str of an object." I was trying to cast floats as strings; not wise.
- I didn't account for cases in which x and y might have a different number of digits, or have an odd number of digits (the latter case would lead to the former). I assumed that, because 64 is a power of 2, both integers would break down perfectly into even halves all the way down to the base case. My assumption is incorrect. Take a simple 4-digit example: 6702. Breaking this down, we get 67 and...2? They're uneven. I initially took n to be the number of digits in x; this is completely arbitrary but harmless. The issue comes with n and n/2. Recall that this is the Karatsuba formula:

$$ x \cdot y = 10^{n}ac + 10^{n/2}[(a+b)(c+d) - ac - bd] + bd$$

- If x and y don't contain an even number of digits, however, then the first expression ac shouldn't necessarily be raised to n. Let's say that x is 3 digits long. Then $n=3$ and $n/2=1$. Thus, we really shouldn't raise ac to $10^n$ ($10^3$), but $10^{n/2*2}$ instead, which is $10^2$.

In [2]:
# My version. Makes error 3, but not 2. 
from math import log10, floor, ceil
def katsubaraMult(x, y):
    # base case: x and y are 2 digits each.
    if x < 100 and y < 100:
        return x * y
    else:
        # gets number of digits.
        n = len(str(x))
        # calculates 10^n
        raised = 10 ** n
        raised2 = 10 ** (n // 2)
        a = int(x / raised2)
        b = int(x % raised2)
        c = int(y / raised2)
        d = int(y % raised2)
        ac = katsubaraMult(a, c)
        bd = katsubaraMult(b, d)
        abcd = katsubaraMult(a + b, c + d)
        return raised * ac + raised2 * (abcd - ac - bd) + bd

# the version I copied. Originally made error 2, but I've corrected float division / to integer division //. 
def karatsubaInt(x,y):
	if len(str(x)) == 1 and len(str(y)) == 1:
		return x * y
	else:
		n = max(len(str(x)), len(str(y)))
		nby2 = n // 2

		a = x // 10**(nby2)
		b = x % 10**(nby2)
		c = y // 10**(nby2)
		d = y % 10**(nby2)

		ac = karatsubaInt(a,c)
		bd = karatsubaInt(b,d)
		ad_plus_bc = karatsubaInt(a+b,c+d) - ac - bd

    	# this little trick, writing n as 2*nby2 takes care of both even and odd n
		prod = ac * 10**(2*nby2) + (ad_plus_bc * 10**nby2) + bd

		return prod

# my version, with error 3 corrected. 
def katsubaraMult2(x, y):
    # base case: x and y are 2 digits each.
    if x < 100 and y < 100:
        return x * y
    else:
        # gets number of digits.
        n = len(str(x))
        # calculates 10^n
        raised2 = 10 ** (n // 2)
        raised = 10 ** (n // 2 * 2)
        a = x // raised2
        b = x % raised2
        c = y // raised2
        d = y % raised2
        ac = katsubaraMult2(a, c)
        bd = katsubaraMult2(b, d)
        abcd = katsubaraMult2(a + b, c + d)
        return raised * ac + raised2 * (abcd - ac - bd) + bd

x = 3141592653589793238462643383279502884197169399375105820974944592
y = 2718281828459045235360287471352662497757247093699959574966967627

a = katsubaraMult(x, y)
print(str(a))
b = karatsubaInt(x, y)
print(str(b))
c = katsubaraMult2(x, y)
print(str(c))

8540949146687105411422007344106059852471069075343497875612961471396184676713802643773539505481409950827382176559558084925815384
8539734222673567065463550869546574495034888535765114961879601127067743044893204848617875072216249073013374895871952806582723184
8539734222673567065463550869546574495034888535765114961879601127067743044893204848617875072216249073013374895871952806582723184


Some things that I thought might have been issues but turned out to be non-issues.
- After creeping around the course forum from terms past, I learned that a lot of people (primarily people working in non-Python languages) had to convert their integers into strings because the language didn't support precise string multiplication after a certain point. I converted my function to take in and output strings. I changed the input/output type to integer, and it didn't make a difference.
- The base case. My original base case was: x AND y are sufficiently small (2-digits long, but 1 digit is fine too). The other solution's base case was: x OR y is 1-digit long. It actually doesn't matter which one we set as the base case -- although we should use the AND case if we want to stay true to the spirit of Karatsuba multiplication. For instance, if x is 5 (one digit) and y is 21 (two digits), we get the same answer regardless of whether we take their product directly (x * y) or we break x into $a=0, b=5$ and y into $c=2, d=1$ and apply one more layer of Karatsuba: 105.

## Week 2: Divide and Conquer
02/06/17

The first half of this week focuses on the Divide and Conquer approach to algorithms, which consists of three main steps:

1. Divide the problem into smaller sub-problems (conceptually or literally).
2. Conquer via recursive calls. Do some clean-up.
3. Combine solutions of the subproblems into one for the original problem.

According to the prof, the most ingenuity/creativity often happes at steps 1 and 3: how we divide the problem and how we combine solutions.

With that, I'm going to implement an algorithm for counting inversions in an array. An inversion is defined as a pair of items at indices x, y for which array[x] > array[y] (i.e., the two items are out of order). We count inversions by counting the number of pairs in an array that are out of order. For a given array, the number of inversions is going to be between 0 (sorted array) or ${n\choose 2}$ (all possible pairs of items -- this is the number of inversions for an array that is completely reversed).

This algorithm 'piggybacks' off of merge sort; the merge step is also used to count inversions. The key insight is this: when we take an item from the right sub-array and place it in our newly sorted array, the number of inversions that item is a part of is equal to the number of items in the left sub-array that remain.

In [7]:
# Inversion count main function.
def invCount(arr):
    if len(arr) < 2:
        return [arr, 0]
    mid = len(arr) // 2
    first, leftInv = invCount(arr[:mid])
    second, rightInv = invCount(arr[mid:])
    both, splitInv = invCountMerge(first, second)
    return [both, leftInv + rightInv + splitInv]

# Merge helper that merges already-sorted sub-arrays.
def invCountMerge(first, second):
    merged = []
    inversions = 0
    # while at least one of these aren't empty.
    while first or second:
        if first and second:
            if first[0] <= second[0]:
                merged.append(first[0])
                first = first[1:]
            else:
                # if item in second list comes before first.
                merged.append(second[0])
                second = second[1:]
                inversions += len(first)
        # only first contains more items.
        elif first:
            merged.append(first[0])
            first = first[1:]
        # only second contains more items.
        else:
            merged.append(second[0])
            second = second[1:]
    return [merged, inversions]

#arr = [5, 6, 2, 1, 8, 3, 7, 4, 2]
arr = [9, 2, 3, 4, 5, 6, 7, 8, 1]
print(invCount(arr)[1])

# code I used to open the file and do programming assignment 2 (idk that it actually works in jupyter):
# with open("inversions_test.txt", "r") as f:
#     arr = [int(line.strip()) for line in f]

inversions = invCount(arr)[1]
print(str(inversions))

15
2407905288


### Some additional divide-and-conquer problems for funsies

This week also discussed Strassen's subcubic matrix multiplication algorithm and the Master Method (a way of finding the runtime of divide-and-conquer algorithms), but those are covered in my notes.

Here are some additional theoretical problems for practice. Just because they're theoretical doesn't mean I'm not going to try to implement at least a couple, though.

---

The following problems are for those of you looking to challenge yourself beyond the required problem sets and programming questions. Most of these have been given in Stanford's CS161 course, Design and Analysis of Algorithms, at some point. They are completely optional and will not be graded. While they vary in level, many are pretty challenging, and we strongly encourage you to discuss ideas and approaches with your fellow students on the "Theory Problems" discussion form.

1. You are given as input an unsorted array of n distinct numbers, where n is a power of 2. Give an algorithm that identifies the second-largest number in the array, and that uses at most $n +\log_2 n - 2$ comparisons.
2. You are a given a unimodal array of n distinct elements, meaning that its entries are in increasing order up until its maximum element, after which its elements are in decreasing order. Give an algorithm to compute the maximum element that runs in O(log n) time.
3. You are given a sorted (from smallest to largest) array A of n distinct integers which can be positive, negative, or zero. You want to decide whether or not there is an index i such that A[i] = i. Design the fastest algorithm that you can for solving this problem.
4. You are given an n by n grid of distinct numbers. A number is a local minimum if it is smaller than all of its neighbors. (A neighbor of a number is one immediately above, below, to the left, or the right. Most numbers have four neighbors; numbers on the side have three; the four corners have two.) Use the divide-and-conquer algorithm design paradigm to compute a local minimum with only O(n) comparisons between pairs of numbers. (Note: since there are $n^2$ numbers in the input, you cannot afford to look at all of them. Hint: Think about what types of recurrences would give you the desired upper bound.)

In [36]:
# Problem 1: finding the second-largest number in an unsorted array of n items, where n > 0. idek rn.


def findMax(arr):
    if len(arr) == 1:
        return arr[0]
    # return the larger of two numbers.
    elif len(arr) == 2:
        larger = arr[0] if arr[0] > arr[1] else arr[1]
        return larger
    else:
        mid = len(arr) // 2
        leftMax = findMax(arr[:mid])
        rightMax = findMax(arr[mid:])
        larger = leftMax if leftMax > rightMax else rightMax
        return larger

# Problem 2: given a unimodal array of n distinct elements, find the max element in the array.
# By definition, a unimodal array will never have the max element be at the edge of the array.

def findMaxUnimodal(arr, low, high):
    mid = (high + low) // 2
    # base case: both the item before and the item after the current one are lower; we are at the max.
    if arr[mid] > arr[mid - 1] and arr[mid] > arr[mid + 1]:
        return arr[mid]
    # recursive case: we're on the up slope.
    elif arr[mid] < arr[mid + 1]:
        return findMaxUnimodal(arr, mid + 1, high)
    # recursive: we're on the down slope.
    elif arr[mid] < arr[mid - 1]:
        return findMaxUnimodal(arr, low, mid - 1)

arr = [0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 15, 10, 6, 4, 3, 1, 0, -1, -4]
maximum = findMaxUnimodal(arr, 0, len(arr) - 1)
print(str(maximum))


# Problem 3: Find if there is an item such that arr[i] == i.

def arrI(arr, low, high):
    guess = (low + high) // 2
    negGuess = -(len(arr) - guess)
    print("AREA:", low, high)
    print(guess, negGuess)
    # base case 2: we converge on one value, but still can't find it.
    # TODO: still missing a case, i think, where we go out of range w/ our first 'if'.
    if low > high:
        return False
    # base case 1: we find it.
    elif arr[guess] == guess or arr[guess] == negGuess:
        return True
    # rule out right side of array.
    if arr[guess] >= 0 and arr[guess] > guess:
        return arrI(arr, low, guess - 1)
    # rule out left side of array.
    elif arr[guess] < 0 and arr[guess] < negGuess:
        return arrI(arr, guess + 1, high)
    else:
        # left OR right.
        return arrI(arr, low, guess - 1) or arrI(arr, guess + 1, high)

arr2 = [-5, -2, 1, 5, 6]
arr3 = [-4, -2, 1, 5, 6]
i = arrI(arr2, 0, 4)
print(i)
i2 = arrI(arr3, 0, 4)
print(i2)

15
AREA: 0 4
2 -3
AREA: 0 1
0 -5
True
AREA: 0 4
2 -3
AREA: 0 1
0 -5
AREA: 0 -1
-1 -6
AREA: 1 1
1 -4
AREA: 1 0
0 -5
AREA: 2 1
1 -4
AREA: 3 4
3 -2
AREA: 3 2
2 -3
False


## Week 3: Quicksort

02/12/18

We cover the quicksort algorithm -- its design, analysis, and issues of probability related to pivot selection and randomness in algorithms in general. I've learned quicksort at least twice already, but this is the first time I'm learning these two things about quicksort:
- there's a way to partition the array (and do the whole sort, in fact) without taking up any additional memory. The entirety of quicksort can be performed in place, which makes it more memory efficient than mergesort (slightly sad because mergesort is a favorite of mine).
- the mathematical reasoning behind why a random pivot is a good pivot and yields $O(nlogn)$ runtime.

I still have yet to learn the second thing, as the probability section comes later in the week. However, I can certainly implement the partition subroutine at this point. Let's implement quicksort now! We'll default to selecting the first item in the sub-array as our pivot (lol) and write code that partitions in place. Pretty cool.

In [51]:
# Quicksort algorithm, with in-place partitioning but without random pivot selection.
def quickSortFull(arr, left, right):
    # if there's 1 or 0 items we're looking at
    if right - left < 2:
        return arr
    
    pivot = arr[left]
    # i = divider btw < and > pivot (arr[i] is the first element > pivot)
    i = left + 1
    # j = divider btw partitioned and unpartitioned (arr[j] is the first unpartitioned elem)
    for j in range(left + 1, right):
        if arr[j] < pivot:
            toSwap = arr[i]
            arr[i] = arr[j]
            arr[j] = toSwap
            i += 1
    # swap pivot with right most elem < pivot
    arr[left] = arr[i - 1]
    arr[i - 1] = pivot
    
    # recursively sort left side in place.
    arr = quickSort(arr, left, i - 1)
    # recursively sort right side in place.
    arr = quickSort(arr, i, right)
    return arr

# Quicksort algorithm, with partitioning subroutine placed in sep. function.
def quickSort(arr, left, right):
    # if there's 1 or 0 items we're looking at
    if right - left < 2:
        return arr

    # partition subroutine places elem < pivot to its left and elem > pivot to its right.
    arr, pivotIndex = partition(arr, left, right)
    # recursively sort left side in place.
    arr = quickSort(arr, left, pivotIndex)
    # recursively sort right side in place.
    arr = quickSort(arr, pivotIndex + 1, right)
    return arr

# partitions around the first element in subarray.
def partition(arr, left, right):
    pivot = arr[left]
    # i = divider btw < and > pivot (arr[i] is the first element > pivot)
    i = left + 1
    # j = divider btw partitioned and unpartitioned (arr[j] is the first unpartitioned elem)
    for j in range(left + 1, right):
        if arr[j] < pivot:
            toSwap = arr[i]
            arr[i] = arr[j]
            arr[j] = toSwap
            i += 1
    # swap pivot with right most elem < pivot
    arr[left] = arr[i - 1]
    arr[i - 1] = pivot
    return (arr, i - 1)

arr = [3, 8, 2, 5, 1, 4, 7, 6]
quickSort(arr, 0, 8)
quickSortFull(arr, 0, 8)

[1, 2, 3, 4, 5, 6, 7, 8]

### Counting comparisons in Quicksort
02/15/18

The homework for this week involves counting the number of comparisons quickSort makes when using a particular pivot selection method. There are three that the assignment asks us to implement: choosing the first element as the pivot (naive), choosing the last element (equally naive), and choosing the median of the first, last, and middle element of the subarray (decidedly less naive). As we can see below, the third makes drastically fewer comparisons than the first two: 

1. 162085
2. 164123
3. 138382

It took me a while to get these numbers, though. Along the way, I made a couple (in hindsight) really quite silly mistakes. Here they are, a reminder that I should never do something this silly again:
- I got the first one wrong because while I counted the number of comparisons each *sub*array made, I totally neglected to count the very first level of comparisons at the root -- n-1 (in our case, 10,000-1) comparisons. Clever.
- Finding the middle index of a subarray for the last pivot selection method (median of first, middle, last). Who knew that (left - right) // 2 wasn't the right way to go about it? This code would be fine if I were trying to find half of the *difference* between the first and the last elements. But I wasn't. I was looking for the middle element, which (duh) we find by calculating (left *+* right) // 2. The luxury of Python list concatenation is making me lazy, because I kept expecting that every single subarray just started at index 0. Nope, they don't. Thank goodness I'm done.

In [3]:
# Quicksort algorithm, that counts comparisons
def quickSortComparisons(arr, left, right):
    # if there's 1 or 0 items we're looking at
    comparisons = max(right - left - 1, 0)
    if right - left < 2:
        return (arr, 0)

    # partition subroutine places elem < pivot to its left and elem > pivot to its right.
    arr, pivotIndex = partitionMedian(arr, left, right)

    # recursively sort left side in place.
    arr, compLeft = quickSortComparisons(arr, left, pivotIndex)
    # recursively sort right side in place.
    arr, compRight = quickSortComparisons(arr, pivotIndex + 1, right)
    return (arr, comparisons + compLeft + compRight)

# partitions around the first element in subarray.
def partitionFirst(arr, left, right):
    pivot = arr[left]
    # i = divider btw < and > pivot (arr[i] is the first element > pivot)
    i = left + 1
    # j = divider btw partitioned and unpartitioned (arr[j] is the first unpartitioned elem)
    for j in range(left + 1, right):
        if arr[j] < pivot:
            toSwap = arr[i]
            arr[i] = arr[j]
            arr[j] = toSwap
            i += 1
    # swap pivot with right most elem < pivot
    arr[left] = arr[i - 1]
    arr[i - 1] = pivot
    return (arr, i - 1)

# partitions around the last element in subarray.
def partitionLast(arr, left, right):
    # select pivot as last item in subarray.
    pivot = arr[right - 1]
    # swap first item with pivot.
    arr[right - 1] = arr[left]
    arr[left] = pivot
    
    # i = divider btw < and > pivot (arr[i] is the first element > pivot)
    i = left + 1
    # j = divider btw partitioned and unpartitioned (arr[j] is the first unpartitioned elem)
    for j in range(left + 1, right):
        if arr[j] < pivot:
            toSwap = arr[i]
            arr[i] = arr[j]
            arr[j] = toSwap
            i += 1
    # swap pivot with right most elem < pivot
    arr[left] = arr[i - 1]
    arr[i - 1] = pivot
    return (arr, i - 1)

from statistics import median
from math import floor
# partitions around the median of the first, last, and middle element.
def partitionMedian(arr, left, right):
    # choose pivot.
#     if right - left > 2:
#         possibleIndexes = [left, (right + left - 1) // 2, right - 1]
#         possiblePivots = [arr[i] for i in possibleIndexes]
#         pivotIndex = list(filter(lambda i: arr[i] == median(possiblePivots), possibleIndexes))[0]
#     else:
#         pivotIndex = left if arr[left] < arr[right - 1] else right - 1
    # arguably less convoluted: get the maxes of each pair, then take the min to get the second highest value.
    # could also use statistics.median, but meh.
    pivotCandidates = {arr[left] : left, arr[(right+left-1)//2] : (right+left-1)//2, arr[right-1]: right-1}
    pivotIndex = pivotCandidates[min(max(arr[left], arr[(right+left-1)//2]), max(arr[left], arr[right-1]), 
                                     max(arr[(right+left-1)//2], arr[right-1]))]

    pivot = arr[pivotIndex]
    arr[pivotIndex] = arr[left]
    arr[left] = pivot
    
    # i = divider btw < and > pivot (arr[i] is the first element > pivot)
    i = left + 1
    # j = divider btw partitioned and unpartitioned (arr[j] is the first unpartitioned elem)
    for j in range(left + 1, right):
        if arr[j] < pivot:
            toSwap = arr[i]
            arr[i] = arr[j]
            arr[j] = toSwap
            i += 1
    # swap pivot with right most elem < pivot
    arr[left] = arr[i - 1]
    arr[i - 1] = pivot
    return (arr, i - 1)

with open("quicksort_test.txt", "r") as f:
    arr = [int(line.strip()) for line in f]

quickSortComparisons(arr, 0, len(arr))

([1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  

## Week 4: Randomized selection and smallest cut graphs
02/20/18

We start the last week of this course (that was fast!) by discussing yet another randomized algorithm: a selection algorithm that computes the $i^{th}$ smallest element in an unsorted array in linear time. Cool, right? 

For instance, say we want to find the median of an array (where the median is the $\frac{n}2$nd smallest element in an even array, $\frac{n+1}{2}$ in an odd array). We could do this by *reducing* the problem to one of sorting. We sort the algorithm using merge sort or quicksort, then simply access the $i^{th}$ element. This takes $O(nlogn)$ time. It turns out that selection is easier than sorting; we can do even better. We can find the $i^{th}$ order statistic in linear time using a special ingredient; randomness. (Interesting tidbit: it's a established fact that the fastest sorting can ever get is $O(nlogn)$. There is no such thing as a linear sorting algorithm. Quite curious about how to prove this, actually.)

The idea is to pivot and partition in the same way as quicksort. However, instead of getting us closer to a sorted array, the act of partioning lets us narrow down the search space for where our target element is. Let's say we want to find the 5th order statistic (5th smallest item) in an array, and we pick a pivot that happens to be the 3rd smallest. Now we know that the thing we're looking for is going to be in the second sub-array. We do one recursive call on that subarray (vs. two in quick sort) and keep searching until we find it!

With that, I'm going to implement this randomized selection algorithm!

In [6]:
import random

# randomized selection algorithm.
# finds and returns the ith smallest element of an unsorted array.
# assumes the array is at least i elements long; if i is too large, we return the largest item in the array.
def randomSelect(arr, target):
    if len(arr) == 1:
        return arr[0]
    # select pivot
    pivotIndex = random.randrange(len(arr))
    pivot = arr[pivotIndex]
    # switch pivot with item at [0]
    arr[pivotIndex] = arr[0]
    arr[0] = pivot
    i = 1
    for j in range(1, len(arr)):
        # if item > pivot, simply increment j
        # if item < pivot, switch the item with the item at i
        if arr[j] < pivot:
            temp = arr[j]
            arr[j] = arr[i]
            arr[i] = temp
            i += 1
    # on the off chance that our pivot is what we're looking for, return
    if target == i:
        return pivot
    # put pivot between < and > subarrays
    arr[0] = arr[i-1]
    arr[i-1] = pivot
    if target < i:
        return randomSelect(arr[:i-1], target)
    elif target > i:
        return randomSelect(arr[i:], target - i)

arr = [10, 8, 2, 4]
randomSelect(arr, 3)

8

### Graphs!

Graphs are complicated. But I guess they're pretty simple, too. Here's a formal definition of a graph, just for kicks:

Graphs consist of vertices (aka nodes) V and edges (sometimes known as arcs) E. The measurement of the size of a graph depends on the number n of nodes and m of edges.

We can group graphs a number of different ways, including:
- There are *connected* graphs ("all in one piece") graphs and *unconnected* graphs. 
- Graph edges can be *undirected* (an edge from u and v means there's an edge from v to u) or *directed* (some edges only go from u to v, and not vice-versa).
- Graphs can be characterized as being *sparse* or *dense*. The line between a sparse and dense graph is blurry, but broadly, the number of edges m in a *sparse* graph is closer to $O(n)$, whereas the number of edges in a dense graph is closer to $O(n^2)$.

### Minimum Graph Cuts: the Problem

A cut of a graph (V, E) is a partition of the graph into two non-empty sets A and B. The minimum cut of a graph is the grouping of a graph into two parts A and B such that A and B have the fewest number of crossing edges as possible. This can be useful for identifying bottlenecks in networks, as a first-order heuristic for detecting communities in social networks, and for edge detection in image segmentation. Cool, right? So how does one find the minimum cut of a graph?

### Minimum Graph Cuts: Karger's Random Contraction Algorithm

